In [56]:
# Notebook setup
%load_ext autoreload
%autoreload 2
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from cerebra_atlas_python.utils import setup_logging
setup_logging(level="DEBUG")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
# Imports
import os
import os.path as op
import logging

import mne
import numpy as np
from mne import bem

Freesurfer and MNE must be installed 

## Constants

Datasets:
- [CerebrA](https://gin.g-node.org/anamanera/CerebrA/src/master/): $CEREBRA_DIR
- [ICBM 2009c Nonlinear Symmetric [NIFTI]](https://nist.mni.mcgill.ca/icbm-152-nonlinear-atlases-2009/): $ICBM_DIR

In [58]:
# Datasets
ICBM_DIR = op.join(os.getenv("DATASETS_DIR"),"ICBM_2009c_Nonlinear_Symmetric/mni_icbm152_nlin_sym_09c_nifti")
CEREBRA_DIR = op.join(os.getenv("DATASETS_DIR"),"Cerebra")

# FreeSurfer
FS_SUBJECTS_DIR = os.getenv("SUBJECTS_DIR")
NEW_SUBJECT_NAME = "icbm152" # Used by FreeSurfer
NEW_FS_SUBJECT_DIR = op.join(FS_SUBJECTS_DIR,NEW_SUBJECT_NAME)
BEM_DIR = op.join(NEW_FS_SUBJECT_DIR, "bem")

# Output dir
CEREBRA_DATA_DIR = "../cerebra_atlas_python/cerebra_data"
HEAD_MRI_T_PATH = op.join(CEREBRA_DATA_DIR,"head_mri_t.fif")
FIDUCIALS_PATH = op.join(BEM_DIR,"icbm152-fiducials.fif")
INFO_PATH = op.join(CEREBRA_DATA_DIR,"info.fif")
MONTAGE_KIND = "GSN-HydroCel-129"
MONTAGE_HEAD_SIZE = 0.1025

# MNIAverage (icbm152)

#### Run fs recon-all

In [59]:
subject_dir = op.join(FS_SUBJECTS_DIR, NEW_SUBJECT_NAME)
if op.exists(subject_dir):
    logging.info("%s FreeSurfer subject already exists, skipping recon-all", NEW_SUBJECT_NAME)
else:
    logging.info("Running recon-all for %s", NEW_SUBJECT_NAME)
    !recon-all -subjid icbm152 -i $ICBM_DIR/mni_icbm152_t1_tal_nlin_sym_09c.nii -all
    !mkdir -p $CEREBRA_DATA_DIR/subjects/$NEW_SUBJECT_NAME/mri
    !cp $NEW_FS_SUBJECT_DIR/mri/T1.mgz $CEREBRA_DATA_DIR/subjects/$NEW_SUBJECT_NAME/mri/T1.mgz
    !cp $NEW_FS_SUBJECT_DIR/mri/wm.asegedit.mgz $CEREBRA_DATA_DIR/subjects/$NEW_SUBJECT_NAME/mri/wm.mgz

INFO:root:icbm152 FreeSurfer subject already exists, skipping recon-all


#### Run mne.bem.make_watershed_bem 
BEM surfaces were generated using the FreeSurfer watershed algorithm through MNE's `mne.bem.make_watershed_bem` and then manually edited so that all inner surfaces are contained within the outer surfaces as explained [here](https://mne.tools/stable/auto_tutorials/forward/80_fix_bem_in_blender.html).

In [60]:
if op.exists(BEM_DIR):
    logging.info("BEM surfaces for subject %s already exist, skipping make_watershed_bem", NEW_SUBJECT_NAME)
else: 
    logging.info("Running make_watershed_bem for %s", NEW_SUBJECT_NAME)
    bem.make_watershed_bem(NEW_SUBJECT_NAME)
    !mkdir -p $CEREBRA_DATA_DIR/subjects/$NEW_SUBJECT_NAME/bem
    !cp $NEW_FS_SUBJECT_DIR/bem/* $CEREBRA_DATA_DIR/subjects/$NEW_SUBJECT_NAME/bem

INFO:root:BEM surfaces for subject icbm152 already exist, skipping make_watershed_bem


<div style="display:flex;align-items:center;justify-content:center;background-color:black;padding:25px;flex-direction:column"><img src="../images/bem_manual_edit.png" alt="BEM MANUAL EDIT" width=50%></img><br/><small>Manual editing of BEM surfaces produced by make_watershed_bem</small></div>

#### Create info (head size)

In [61]:
# Remove chin and neck
# TODO: Insert reference
electrode_names = [
        "E1",
        "E2",
        "E3",
        "E4",
        "E5",
        "E6",
        "E7",
        "E8",
        "E9",
        "E10",
        "E11",
        "E12",
        "E13",
        "E14",
        "E15",
        "E16",
        "E18",
        "E19",
        "E20",
        "E21",
        "E22",
        "E23",
        "E24",
        "E25",
        "E26",
        "E27",
        "E28",
        "E29",
        "E30",
        "E31",
        "E32",
        "E33",
        "E34",
        "E35",
        "E36",
        "E37",
        "E38",
        "E39",
        "E40",
        "E41",
        "E42",
        "E43",
        "E44",
        "E45",
        "E46",
        "E47",
        "E50",
        "E51",
        "E52",
        "E53",
        "E54",
        "E55",
        "E57",
        "E58",
        "E59",
        "E60",
        "E61",
        "E62",
        "E64",
        "E65",
        "E66",
        "E67",
        "E69",
        "E70",
        "E71",
        "E72",
        "E74",
        "E75",
        "E76",
        "E77",
        "E78",
        "E79",
        "E80",
        "E82",
        "E83",
        "E84",
        "E85",
        "E86",
        "E87",
        "E89",
        "E90",
        "E91",
        "E92",
        "E93",
        "E95",
        "E96",
        "E97",
        "E98",
        "E100",
        "E101",
        "E102",
        "E103",
        "E104",
        "E105",
        "E106",
        "E108",
        "E109",
        "E110",
        "E111",
        "E112",
        "E114",
        "E115",
        "E116",
        "E117",
        "E118",
        "E120",
        "E121",
        "E122",
        "E123",
        "E124",
        "Cz",
    ]

In [62]:
if op.exists(INFO_PATH):
    logging.info("BEM surfaces for subject %s already exist, skipping make_watershed_bem", NEW_SUBJECT_NAME)
else:
    from cerebra_atlas_python.utils import get_standard_montage
    logging.info("Creating info file for %s", NEW_SUBJECT_NAME)
    montage = get_standard_montage(electrode_names,kind=MONTAGE_KIND,head_size=MONTAGE_HEAD_SIZE)
    kept_ch_names = montage.ch_names
    info = mne.create_info(kept_ch_names, 5000, ch_types='eeg', verbose=False)
    info.set_montage(montage)
    info.save(INFO_PATH)


INFO:root:BEM surfaces for subject icbm152 already exist, skipping make_watershed_bem


#### Manually align fiducials
Produces 

subjects/icbm152/bem/icbm152-fiducials.fif 
and
head_mri_t.fif

In [63]:
# Save fiducials to $bem_dir/icbm152-fiducials.fif
# Save fiducials to $./head_mri_t.fif
if not op.exists(FIDUCIALS_PATH) or not op.exists(HEAD_MRI_T_PATH):
    import mne
    mne.gui.coregistration()
logging.info("Fiducials and head to mri transform already exists %s and %s", FIDUCIALS_PATH, HEAD_MRI_T_PATH)

INFO:root:Fiducials and head to mri transform already exists /home/carlos/Datasets/subjects/icbm152/bem/icbm152-fiducials.fif and ../cerebra_atlas_python/cerebra_data/head_mri_t.fif


In [79]:
from cerebra_atlas_python.mni_average import MNIAverage

mni_average = MNIAverage()
mni_average.head_mri_t

DEBUG:root:Value for variable cerebra_data_path not provided in config.ini[MNIAverage]. Defaulting to cerebra_data_path=/home/carlos/Carlos/cerebra_atlas_python/cerebra_atlas_python/cerebra_data
INFO:root:Loading boundary element model from disk | MNIAverage_bem_0.33_0.0042_0.33_ico_4
DEBUG:root:Fiducials coordinate frame: 5 (FIFFV_COORD_MRI)


<Transform | head->MRI (surface RAS)>
[[ 0.99998409  0.00051499 -0.00561859 -0.00032736]
 [-0.00093964  0.99711949 -0.07584107 -0.00204062]
 [ 0.00556335  0.07584514  0.99710411 -0.05805472]
 [ 0.          0.          0.          1.        ]]

In [81]:
mne.dig_mri_distances(mni_average.info, mni_average.head_mri_t, mni_average.subject_name).sum()
mne.viz.plot_alignment(bem=mni_average.bem, mri_fiducials=mni_average.fiducials, trans=mni_average.head_mri_t, show_axes=True,dig=True, info = info)

DEBUG:qdarkstyle:Reading QSS file in: :qdarkstyle/dark/darkstyle.qss
INFO:qdarkstyle:QSS file successfully loaded.
DEBUG:qdarkstyle:Checking patches for being applied.
INFO:qdarkstyle:Found version patches to be applied.
INFO:qdarkstyle:Found application patches to be applied.


# CerebrA 


In [66]:
# Make sure its good until here
# model = mne.make_bem_model(
#     subject="icbm152",
#     subjects_dir=FS_SUBJECTS_DIR,
#     ico=4,
#     conductivity=(0.33, 0.0042, 0.33),
# )
# bem = mne.make_bem_solution(model)
# 

In [67]:
# cerebra.

In [68]:
# mne.viz.plot_alignment(bem=bem, mri_fiducials=fiducials, trans=head_mri_t, show_axes=True,dig=True, info = info)

In [69]:
plot_alignment(bem=cerebra.mni_average.bem,mri_fiducials=cerebra.mni_average.fiducials, trans=trans, show_axes=True,dig=True, info = info, src=cerebra.src_space)

NameError: name 'plot_alignment' is not defined